# Solve the multidimensional knapsack problem with quantum annealers
## No recomiendo correr todo el código de seguido.
Los pasos que yo he seguido han sido:

1.   Seleccionar el fichero que queria trabajar (ver en el índice)
2.   Compilar el QUBO relacionado.
3.   Ejecutar la simulación (o en DWave) para resolverlo.



Vamos a hacer unas instalaciones primero, para prepara el google colab.

In [ ]:
!pip install dwave-ocean-sdk networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.9/158.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00


Now we need to configure the environment (press 'enter' to use the default options). If you have an API key to use D-Wave systems, you can enter the key at this stage.

Import the required libraries.

In [ ]:
from dimod import Binary, ExactSolver
from networkx import Graph, draw
from dwave.samplers import SimulatedAnnealingSampler, SteepestDescentSampler
from dwave.system import DWaveSampler, EmbeddingComposite

from google.colab import files
import os
from dimod import BinaryQuadraticModel
import math

Para importar los ficheros .txt al google colab:

*Nota:* Algunos ficheros han con las filas cortadas, asi que he reagrupado los datos (los he subido junto con el notebook).


In [ ]:
uploaded = files.upload()

Saving md-knap5.txt to md-knap5 (1).txt


## Formulación del problema de la mochila multidimensional para QUBO

Nuestro problema es de la forma:

$\max \sum_{i=1}^nc_ix_i $

s.a.

$\sum_{i=1}^nx_iw_{ij} \leq W_j$, $\forall j \in \{1,2,\dots,m\}$

Queremos pasar la restricciones del problema a la funcion objetivo a modo de penalización cuando alguna no se cumpla, para ello vimos en las clases que podemos hacer los siguientes cambios.

$\sum_{i=1}^nx_iw_{ij} \leq W_j ≡\sum_{i=1}^nx_iw_{ij} -  W_j\leq 0 ≡ \sum_{i=1}^nx_iw_{ij} -  W_j + v_j = 0$ para una variable $v_j \geq 0$. Tomando como cota superior de $v_j$ el valor -min($\sum_{i=1}^nx_iw_{ij} -  W_j$) o alguna aproximación para ahorrarnos cálculos, nuestro problema de optimizacion se puede reescribir como QUBO de manera:

$\max\sum_{i=1}^nc_ix_i - P \sum_{j=1}^m(\sum_{i=1}^nx_iw_{ij} -  W_j + v_j)^2$

Para manderle el problema a DWave, lo pasamos a minimizar:

$-\min\sum_{i=1}^n-c_ix_i + P \sum_{j=1}^m(\sum_{i=1}^nx_iw_{ij} -  W_j + v_j)^2$

Para $P$ una constante de la forma $P = \sum_{i=1}^n |c_i|$.

*Nota:* si queremos ser precavidos podemos tomar $P + 1$ para evitarnos el caso de que las Feasible Solutions tan solo se encuentren en el extremo superior $\sum_{i=1}^n \max(0,c_i)$ y por tanto pueden llegar a coincider con las Infeasible Solution con penalización $P$.


# Seleccionamos el archivo con el cual queremos trabajar:

In [ ]:
# Selecciona el archivo que deseas leer
archivo_seleccionado = input("Selecciona el archivo con el cual quieres trabajar:")
#Opciones: md-knap1.txt // md-knap2.txt // md-knap3.txt // md-knap4.txt // md-knap5.txt // md-knap6.txt // md-knap7.txt


# Ruta completa del archivo en el entorno local
ruta_archivo = os.path.join('/content', archivo_seleccionado)

# Lee el archivo y procesa los datos
try:
    with open(ruta_archivo, 'r') as f:
        datos = []
        for linea in f:
            # Divide cada línea en una lista de números (convertir a float o int según necesites)
            fila = [float(num) for num in linea.split()]
            datos.append(fila)

    # Muestra los datos leídos
    print("Datos en el archivo seleccionado:")
    for fila in datos:
        print(fila)

except FileNotFoundError:
    print("Archivo no encontrado. Asegúrate de que el nombre es correcto.")


Selecciona el archivo con el cual quieres trabajar:md-knap7.txt
Datos en el archivo seleccionado:
[50.0, 5.0, 16537.0]
[560.0, 1125.0, 300.0, 620.0, 2100.0, 431.0, 68.0, 328.0, 47.0, 122.0, 322.0, 196.0, 41.0, 25.0, 425.0, 4260.0, 416.0, 115.0, 82.0, 22.0, 631.0, 132.0, 420.0, 86.0, 42.0, 103.0, 215.0, 81.0, 91.0, 26.0, 49.0, 420.0, 316.0, 72.0, 71.0, 49.0, 108.0, 116.0, 90.0, 738.0, 1811.0, 430.0, 3060.0, 215.0, 58.0, 296.0, 620.0, 418.0, 47.0, 81.0]
[40.0, 91.0, 10.0, 30.0, 160.0, 20.0, 3.0, 12.0, 3.0, 18.0, 9.0, 25.0, 1.0, 1.0, 10.0, 280.0, 10.0, 8.0, 1.0, 1.0, 49.0, 8.0, 21.0, 6.0, 1.0, 5.0, 10.0, 8.0, 2.0, 1.0, 0.0, 10.0, 42.0, 6.0, 4.0, 8.0, 0.0, 10.0, 1.0, 40.0, 86.0, 11.0, 120.0, 8.0, 3.0, 32.0, 28.0, 13.0, 2.0, 4.0]
[16.0, 92.0, 41.0, 16.0, 150.0, 23.0, 4.0, 18.0, 6.0, 0.0, 12.0, 8.0, 2.0, 1.0, 0.0, 200.0, 20.0, 6.0, 2.0, 1.0, 70.0, 9.0, 22.0, 4.0, 1.0, 5.0, 10.0, 6.0, 4.0, 0.0, 4.0, 12.0, 8.0, 4.0, 3.0, 0.0, 10.0, 0.0, 6.0, 28.0, 93.0, 9.0, 30.0, 22.0, 0.0, 36.0, 45.0, 13.0, 

###Compilar el código para definir el QUBO del archivo seleccionado anteriormente.

In [ ]:
#Partiendo de nuestro conjunto de 'datos' seleccionado, definimos:
n = int(datos[0][0]) #Numero de variables
m = int(datos[0][1]) #Numero de restricciones, j=1...m
solopti = datos[0][2] #Para comparar solucion al final
ci = datos[1] #Vector de coeficientes c_i
def wi(j): #Vector de coeficientes w_ij
  return datos[j+2]

Wj = datos[-1] #Vector de coeficientes W_j
xi = [Binary(f'x{i+1}') for i in range (0,n)] #variables x_i, i=1...n
#==================#
#==================#

# definimos la funcion objetivo  \sum -xici
def fobj():
  return -sum([(xi[i]*ci[i]) for i in range (0,n)])

# definimos el valor de la constante P
P = sum([abs(ci[i]) for i in range (0,n)]) +1

# Calculamos C = -min( sum x_i w_ij - W_j ) para obtener la expresion de 0<= v_j <= c en binario
#para ello como w_ij >= 0, tomaremos C = W_j
C = [int(Wj[j]) for j in range (0,m)]

min_n = [int(math.log2(num)) + 1 for num in C] #minimo exponenete para expresar cada W_j en binario
yij =[[Binary(f'y{i+1,j+1}') for i in range (0,min_n[j])] for j in range (0,m)] #variables y_ij, i=1...n  j=1...m
vij =[[Binary(f'v{i+1,j+1}') * 2 **(i) for i in range (0,min_n[j])] for j in range (0,m)] #variables v_ij, i=1...n  j=1...m

# definimos qubo = fobj + P * sum_j[sum_i x_i w_ij - W_j + v_ij]**2 (para v_ij expresion en binario)
def get_QUBO():
  return fobj()+sum([int(P)*((sum([(xi[i]*wi(j)[i]) for i in range (0,n)]) + sum([vij[j][k] for k in range(0,min_n[j])])) - Wj[j])**2 for j in range (0,m)])

qubo = get_QUBO()


# Resultados

## Resultados primer problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  #return EmbeddingComposite(DWaveSampler()) #dwave composer que te adapta el grafo a la topologia de los procesadores de dwave

In [ ]:
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) v(1, 5) ... x6   energy num_oc.
37       0        0       0       0       0       1 ...  1  -3800.0       1
71       1        0       1       0       0       1 ...  0  -2300.0       1
31       0        0       0       0       0       1 ...  0   -600.0       1
51       0        0       1       0       0       0 ...  0   4401.0       1
72       1        0       1       0       1       0 ...  1   4801.0       1
73       0        1       1       1       1       0 ...  0   4901.0       1
34       1        0       1       0       0       0 ...  0   5601.0       1
83       0        1       0       1       0       0 ...  0   5601.0       1
94       0        1       0       1       1       1 ...  0   6201.0       1
62       0        1       0       1       1       0 ...  0   6701.0       1
49       1        0       0       0       0       1 ...  1  10502.0       1
0        1        1       1       1       0       1 ...  1  11502.0       1
36       1  

In [ ]:
first_sample = next(result.samples())
print(first_sample)

{'v(1, 1)': 0, 'v(1, 10)': 0, 'v(1, 2)': 0, 'v(1, 3)': 0, 'v(1, 4)': 0, 'v(1, 5)': 1, 'v(1, 6)': 1, 'v(1, 7)': 0, 'v(1, 8)': 0, 'v(1, 9)': 0, 'v(2, 1)': 1, 'v(2, 10)': 1, 'v(2, 2)': 1, 'v(2, 3)': 1, 'v(2, 4)': 1, 'v(2, 5)': 1, 'v(2, 6)': 1, 'v(2, 7)': 1, 'v(2, 8)': 1, 'v(2, 9)': 0, 'v(3, 1)': 1, 'v(3, 10)': 1, 'v(3, 2)': 1, 'v(3, 3)': 1, 'v(3, 4)': 1, 'v(3, 5)': 0, 'v(3, 6)': 1, 'v(3, 7)': 0, 'v(3, 8)': 1, 'v(3, 9)': 1, 'v(4, 1)': 1, 'v(4, 10)': 1, 'v(4, 2)': 1, 'v(4, 3)': 0, 'v(4, 4)': 0, 'v(4, 5)': 0, 'v(4, 6)': 0, 'v(4, 7)': 1, 'v(4, 8)': 1, 'v(4, 9)': 1, 'v(5, 1)': 0, 'v(5, 10)': 0, 'v(5, 2)': 1, 'v(5, 3)': 0, 'v(5, 4)': 0, 'v(5, 5)': 0, 'v(5, 6)': 0, 'v(5, 8)': 0, 'v(5, 9)': 0, 'v(6, 1)': 0, 'v(6, 2)': 0, 'v(6, 4)': 0, 'v(6, 5)': 0, 'v(6, 6)': 0, 'v(7, 1)': 0, 'v(7, 2)': 0, 'x1': 0, 'x2': 1, 'x3': 1, 'x4': 0, 'x5': 0, 'x6': 1}


<ipython-input-39-c8bdcd21264e>:1: DeprecationWarning: SampleSet.samples() will return an iterable not an iterator in the future
  first_sample = next(result.samples())


Luego para la primera lista hemos obtenido como sol optima:

$x_1 = 0, x_2 = 1,x_3 = 1,x_4 = 0,x_5 = 0,x_6 = 1$

$v_j = 0 ∀ j=1..10$

Con el valor de la funcion objetivo -(-3800) = 3800

Luego la diferencia respecto de la sol optima es de

In [ ]:
solopti - 3800

0

## Resultados segundo problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  #return EmbeddingComposite(DWaveSampler()) #dwave composer que te adapta el grafo a la topologia de los procesadores de dwave
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=1000)
print(result)

    v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) ... x9          energy num_oc.
180       1        1       1       1       0 ...  0         -8009.9       1
710       1        1       1       1       0 ...  1         -5513.1       1
619       0        1       0       0       0 ...  0    -5325.800001       1
373       1        0       1       1       0 ...  0    -5241.200001       1
593       1        0       1       1       0 ...  1    -4761.700001       1
377       1        1       1       1       0 ...  0          4580.1       1
257       0        1       0       0       0 ...  0          7462.9       1
713       0        0       0       0       0 ...  1          8337.5       1
44        0        0       0       0       0 ...  0    16784.399998       1
159       0        1       0       1       0 ...  0    17499.199999       1
196       0        1       0       0       0 ...  0    18035.599998       1
455       0        1       0       0       1 ...  0         18252.9       1
908       1 

In [ ]:
first_sample = next(result.samples())
print(first_sample)

8706.1
{'v(1, 1)': 1, 'v(1, 10)': 1, 'v(1, 2)': 1, 'v(1, 3)': 1, 'v(1, 4)': 0, 'v(1, 5)': 1, 'v(1, 6)': 0, 'v(1, 7)': 0, 'v(1, 8)': 1, 'v(1, 9)': 1, 'v(10, 2)': 0, 'v(2, 1)': 0, 'v(2, 10)': 1, 'v(2, 2)': 0, 'v(2, 3)': 0, 'v(2, 4)': 1, 'v(2, 5)': 0, 'v(2, 6)': 0, 'v(2, 7)': 0, 'v(2, 8)': 0, 'v(2, 9)': 0, 'v(3, 1)': 1, 'v(3, 10)': 0, 'v(3, 2)': 1, 'v(3, 3)': 1, 'v(3, 4)': 1, 'v(3, 5)': 1, 'v(3, 6)': 1, 'v(3, 7)': 0, 'v(3, 8)': 0, 'v(3, 9)': 1, 'v(4, 1)': 1, 'v(4, 10)': 1, 'v(4, 2)': 0, 'v(4, 3)': 0, 'v(4, 4)': 0, 'v(4, 5)': 0, 'v(4, 6)': 0, 'v(4, 7)': 0, 'v(4, 8)': 0, 'v(4, 9)': 0, 'v(5, 1)': 0, 'v(5, 10)': 0, 'v(5, 2)': 1, 'v(5, 3)': 1, 'v(5, 4)': 0, 'v(5, 5)': 0, 'v(5, 6)': 1, 'v(5, 7)': 0, 'v(5, 8)': 0, 'v(5, 9)': 1, 'v(6, 1)': 1, 'v(6, 10)': 0, 'v(6, 2)': 1, 'v(6, 3)': 0, 'v(6, 4)': 1, 'v(6, 5)': 0, 'v(6, 6)': 0, 'v(6, 7)': 1, 'v(6, 8)': 0, 'v(6, 9)': 0, 'v(7, 1)': 1, 'v(7, 10)': 1, 'v(7, 2)': 0, 'v(7, 3)': 0, 'v(7, 4)': 1, 'v(7, 5)': 0, 'v(7, 6)': 0, 'v(7, 7)': 1, 'v(7, 8)': 1, 'v(7

<ipython-input-55-625f970f9865>:2: DeprecationWarning: SampleSet.samples() will return an iterable not an iterator in the future
  first_sample = next(result.samples())


Luego para la lista hemos obtenido como sol optima:

$x1= 1, x10= 0, x2= 1, x3= 1, x4= 0, x5= 1, x6= 1, x7= 1, x8= 1, x9= 0$

$v_j \neq 0 ∀ j=1...m$

Con el valor de la funcion objetivo -(-8009.9) = 8009.9

Entonces la diferencia con la solucion optima es de:

In [ ]:
solopti - 8009.9

696.2000000000007

## Resultados tercer problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  #return EmbeddingComposite(DWaveSampler()) #dwave composer que te adapta el grafo a la topologia de los procesadores de dwave
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=1000)
print(result)

    v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) v(1, 5) ... x9     energy num_oc.
938       0        0       0       1       0       1 ...  0    -3265.0       1
119       1        1       1       1       0       1 ...  1    -2755.0       1
7         0        1       0       1       1       1 ...  0    -2350.0       1
781       0        1       0       1       1       0 ...  0    -2230.0       1
219       1        0       1       0       1       1 ...  0    -2190.0       1
480       0        0       0       1       0       0 ...  0    -2125.0       1
210       1        1       1       0       0       0 ...  1    -1795.0       1
344       0        1       0       0       1       1 ...  1    -1670.0       1
714       0        1       0       0       0       0 ...  1    -1310.0       1
682       0        1       0       1       0       1 ...  0    -1245.0       1
165       1        0       1       0       1       0 ...  1     -970.0       1
637       1        0       1       1       0       1

In [ ]:
first_sample = next(result.samples())
print(first_sample)

{'v(1, 1)': 0, 'v(1, 10)': 0, 'v(1, 2)': 0, 'v(1, 3)': 1, 'v(1, 4)': 0, 'v(1, 5)': 1, 'v(1, 6)': 1, 'v(1, 7)': 1, 'v(1, 8)': 0, 'v(1, 9)': 1, 'v(10, 1)': 0, 'v(10, 2)': 0, 'v(2, 1)': 1, 'v(2, 10)': 1, 'v(2, 2)': 0, 'v(2, 3)': 0, 'v(2, 4)': 1, 'v(2, 5)': 0, 'v(2, 6)': 0, 'v(2, 7)': 1, 'v(2, 8)': 1, 'v(2, 9)': 0, 'v(3, 1)': 0, 'v(3, 10)': 0, 'v(3, 2)': 1, 'v(3, 3)': 0, 'v(3, 4)': 1, 'v(3, 5)': 0, 'v(3, 6)': 0, 'v(3, 7)': 0, 'v(3, 8)': 1, 'v(3, 9)': 1, 'v(4, 1)': 1, 'v(4, 10)': 0, 'v(4, 2)': 1, 'v(4, 3)': 0, 'v(4, 4)': 0, 'v(4, 5)': 1, 'v(4, 6)': 1, 'v(4, 7)': 1, 'v(4, 8)': 0, 'v(4, 9)': 1, 'v(5, 1)': 0, 'v(5, 10)': 0, 'v(5, 2)': 0, 'v(5, 3)': 0, 'v(5, 4)': 0, 'v(5, 5)': 0, 'v(5, 6)': 1, 'v(5, 7)': 0, 'v(5, 8)': 1, 'v(5, 9)': 0, 'v(6, 1)': 0, 'v(6, 10)': 0, 'v(6, 2)': 1, 'v(6, 3)': 1, 'v(6, 4)': 0, 'v(6, 5)': 0, 'v(6, 6)': 0, 'v(6, 7)': 0, 'v(6, 8)': 0, 'v(6, 9)': 0, 'v(7, 1)': 1, 'v(7, 10)': 0, 'v(7, 2)': 0, 'v(7, 3)': 0, 'v(7, 4)': 1, 'v(7, 5)': 1, 'v(7, 6)': 1, 'v(7, 7)': 0, 'v(7, 8)':

<ipython-input-67-c8bdcd21264e>:1: DeprecationWarning: SampleSet.samples() will return an iterable not an iterator in the future
  first_sample = next(result.samples())


Luego para la lista hemos obtenido como sol optima:

$'x1'= 0, 'x10'= 1, 'x11'= 0, 'x12'= 1, 'x13'= 0, 'x14'= 1, 'x15'= 0, 'x2'= 1, 'x3'= 0, 'x4'= 1, 'x5'= 1, 'x6'= 0, 'x7'= 1, 'x8'= 1, 'x9'= 0$

$v_j \neq 0 ∀ j=1..m$

Con el valor de la funcion objetivo -(-3265.0) = 3265.0

Entonces la diferencia con la solucion optima es de:

In [ ]:
solopti - 3265.0

750.0

## Resultados cuarto problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  #return EmbeddingComposite(DWaveSampler()) #dwave composer que te adapta el grafo a la topologia de los procesadores de dwave
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=1000)
print(result)

    v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) v(1, 5) ... x9     energy num_oc.
735       0        0       1       0       1       0 ...  1    -5305.0       1
304       1        0       0       0       1       1 ...  1    -4700.0       1
714       1        0       0       1       1       1 ...  0    -4700.0       1
55        1        1       0       1       0       1 ...  0    -4010.0       1
446       0        1       0       0       1       0 ...  1    -3845.0       1
982       1        1       1       1       0       1 ...  1    -3810.0       1
851       0        1       1       1       0       0 ...  0    -2875.0       1
299       0        1       0       0       1       0 ...  0    -2625.0       1
37        1        0       1       0       0       0 ...  1    -2555.0       1
809       1        1       0       0       1       0 ...  0    -2200.0       1
529       0        1       0       1       1       0 ...  0    -2095.0       1
819       1        1       1       1       1       1

In [ ]:
first_sample = next(result.samples())
print(first_sample)

{'v(1, 1)': 0, 'v(1, 10)': 0, 'v(1, 2)': 1, 'v(1, 3)': 0, 'v(1, 4)': 1, 'v(1, 5)': 0, 'v(1, 6)': 0, 'v(1, 7)': 0, 'v(1, 8)': 0, 'v(1, 9)': 0, 'v(10, 1)': 0, 'v(10, 2)': 0, 'v(2, 1)': 1, 'v(2, 10)': 0, 'v(2, 2)': 0, 'v(2, 3)': 1, 'v(2, 4)': 1, 'v(2, 5)': 1, 'v(2, 6)': 1, 'v(2, 7)': 1, 'v(2, 8)': 1, 'v(2, 9)': 1, 'v(3, 1)': 0, 'v(3, 10)': 1, 'v(3, 2)': 0, 'v(3, 3)': 0, 'v(3, 4)': 0, 'v(3, 5)': 0, 'v(3, 6)': 1, 'v(3, 7)': 0, 'v(3, 8)': 1, 'v(3, 9)': 1, 'v(4, 1)': 1, 'v(4, 10)': 0, 'v(4, 2)': 1, 'v(4, 3)': 1, 'v(4, 4)': 0, 'v(4, 5)': 1, 'v(4, 6)': 0, 'v(4, 7)': 1, 'v(4, 8)': 1, 'v(4, 9)': 1, 'v(5, 1)': 1, 'v(5, 10)': 1, 'v(5, 2)': 0, 'v(5, 3)': 0, 'v(5, 4)': 0, 'v(5, 5)': 0, 'v(5, 6)': 1, 'v(5, 7)': 1, 'v(5, 8)': 1, 'v(5, 9)': 1, 'v(6, 1)': 1, 'v(6, 10)': 0, 'v(6, 2)': 1, 'v(6, 3)': 1, 'v(6, 4)': 1, 'v(6, 5)': 1, 'v(6, 6)': 1, 'v(6, 7)': 0, 'v(6, 8)': 0, 'v(6, 9)': 0, 'v(7, 1)': 0, 'v(7, 10)': 0, 'v(7, 2)': 0, 'v(7, 3)': 0, 'v(7, 4)': 0, 'v(7, 5)': 0, 'v(7, 6)': 0, 'v(7, 7)': 0, 'v(7, 8)':

<ipython-input-9-bc713edfb560>:1: DeprecationWarning: SampleSet.samples() will return an iterable not an iterator in the future
  first_sample = next(result.samples())


Luego para la lista hemos obtenido como sol optima:

$'x1'= 0, 'x10'= 0, 'x11'= 0, 'x12'= 0, 'x13'= 0, 'x14'= 1, 'x15'= 1, 'x16'= 0, 'x17'= 0, 'x18'= 1, 'x19'= 1, 'x2'= 0, 'x20'= 1, 'x3'= 0, 'x4'= 0, 'x5'= 1, 'x6'= 0, 'x7'= 1, 'x8'= 0, 'x9'= 1$

$v_j \neq 0 ∀ j=1..m$

Con el valor de la funcion objetivo -(-5305.0) = 5305.0

Entonces la diferencia con la solucion optima es de:

In [ ]:
solopti - 5305.0

815.0

## Resultados quinto problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  #return EmbeddingComposite(DWaveSampler()) #dwave composer que te adapta el grafo a la topologia de los procesadores de dwave
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=1000)
print(result)

    v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) v(1, 5) ... x9      energy num_oc.
75        0        1       1       0       1       0 ...  1    -10720.0       1
191       0        1       0       0       0       0 ...  1    -10615.0       1
512       0        0       0       1       1       0 ...  0    -10390.0       1
971       0        1       1       0       0       1 ...  0     -7970.0       1
377       1        1       1       1       1       1 ...  0     -7070.0       1
476       0        1       1       0       0       0 ...  1     -6365.0       1
570       1        1       0       0       1       1 ...  1     -5875.0       1
290       0        0       0       0       0       0 ...  0     -4765.0       1
924       1        1       0       1       1       0 ...  1     -4455.0       1
583       1        1       0       1       0       0 ...  1      5296.0       1
77        0        1       0       1       0       1 ...  0      5476.0       1
238       1        0       0       0    

In [ ]:
first_sample = next(result.samples())
print(first_sample)

{'v(1, 1)': 0, 'v(1, 10)': 1, 'v(1, 2)': 1, 'v(1, 3)': 0, 'v(1, 4)': 1, 'v(1, 5)': 0, 'v(1, 6)': 1, 'v(1, 7)': 0, 'v(1, 8)': 0, 'v(1, 9)': 1, 'v(10, 1)': 0, 'v(10, 2)': 0, 'v(10, 5)': 0, 'v(10, 6)': 0, 'v(11, 2)': 0, 'v(2, 1)': 1, 'v(2, 10)': 0, 'v(2, 2)': 0, 'v(2, 3)': 1, 'v(2, 4)': 0, 'v(2, 5)': 1, 'v(2, 6)': 1, 'v(2, 7)': 0, 'v(2, 8)': 1, 'v(2, 9)': 0, 'v(3, 1)': 1, 'v(3, 10)': 0, 'v(3, 2)': 1, 'v(3, 3)': 1, 'v(3, 4)': 1, 'v(3, 5)': 1, 'v(3, 6)': 1, 'v(3, 7)': 1, 'v(3, 8)': 1, 'v(3, 9)': 0, 'v(4, 1)': 1, 'v(4, 10)': 1, 'v(4, 2)': 0, 'v(4, 3)': 1, 'v(4, 4)': 0, 'v(4, 5)': 0, 'v(4, 6)': 1, 'v(4, 7)': 0, 'v(4, 8)': 0, 'v(4, 9)': 1, 'v(5, 1)': 1, 'v(5, 10)': 0, 'v(5, 2)': 0, 'v(5, 3)': 1, 'v(5, 4)': 1, 'v(5, 5)': 0, 'v(5, 6)': 1, 'v(5, 7)': 1, 'v(5, 8)': 1, 'v(5, 9)': 0, 'v(6, 1)': 1, 'v(6, 10)': 1, 'v(6, 2)': 0, 'v(6, 3)': 0, 'v(6, 4)': 0, 'v(6, 5)': 0, 'v(6, 6)': 0, 'v(6, 7)': 0, 'v(6, 8)': 0, 'v(6, 9)': 1, 'v(7, 1)': 1, 'v(7, 10)': 1, 'v(7, 2)': 1, 'v(7, 3)': 1, 'v(7, 4)': 1, 'v(7, 5

<ipython-input-21-bc713edfb560>:1: DeprecationWarning: SampleSet.samples() will return an iterable not an iterator in the future
  first_sample = next(result.samples())


Luego para la lista hemos obtenido como sol optima:

$'x1'= 0, 'x10'= 1, 'x11'= 0, 'x12'= 0, 'x13'= 1, 'x14'= 0, 'x15'= 0, 'x16'= 0, 'x17'= 0, 'x18'= 0, 'x19'= 0, 'x2'= 0, 'x20'= 1, 'x21'= 1, 'x22'= 1, 'x23'= 1, 'x24'= 1, 'x25'= 1, 'x26'= 1, 'x27'= 1, 'x28'= 1, 'x3'= 1, 'x4'= 0, 'x5'= 0, 'x6'= 1, 'x7'= 0, 'x8'= 0, 'x9'= 1$

$v_j \neq 0 ∀ j=1..m$

Con el valor de la funcion objetivo -(-10720.0) = 10720.0

Entonces la diferencia con la solucion optima es de:

In [ ]:
solopti - 10720.0

1680.0

## Resultados sexto problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  #return EmbeddingComposite(DWaveSampler()) #dwave composer que te adapta el grafo a la topologia de los procesadores de dwave
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=1000)
print(result)

    v(1, 1) v(1, 2) v(1, 3) v(1, 4) v(1, 5) v(10, 1) ... x9     energy num_oc.
466       1       1       0       0       0        0 ...  0    -9776.0       1
729       1       0       1       1       1        0 ...  1    -9398.0       1
962       1       0       1       1       1        0 ...  0    -9395.0       1
536       1       0       1       1       1        0 ...  0    -9281.0       1
126       0       1       1       1       1        0 ...  0    -9265.0       1
296       0       0       1       0       0        0 ...  0    -9237.0       1
412       0       0       1       0       1        0 ...  0    -9176.0       1
310       1       1       0       1       1        0 ...  1    -9173.0       1
781       1       1       1       0       0        0 ...  0    -9099.0       1
149       0       1       0       1       1        0 ...  0    -9086.0       1
434       1       0       1       0       1        0 ...  1    -9080.0       1
419       0       1       0       0       1        0

In [ ]:
first_sample = next(result.samples())
print(first_sample)

{'v(1, 1)': 1, 'v(1, 2)': 1, 'v(1, 3)': 0, 'v(1, 4)': 0, 'v(1, 5)': 0, 'v(10, 1)': 0, 'v(10, 5)': 0, 'v(2, 1)': 1, 'v(2, 2)': 1, 'v(2, 3)': 1, 'v(2, 4)': 1, 'v(2, 5)': 0, 'v(3, 1)': 0, 'v(3, 2)': 0, 'v(3, 3)': 1, 'v(3, 4)': 0, 'v(3, 5)': 1, 'v(4, 1)': 0, 'v(4, 2)': 1, 'v(4, 3)': 1, 'v(4, 4)': 1, 'v(4, 5)': 1, 'v(5, 1)': 1, 'v(5, 2)': 0, 'v(5, 3)': 1, 'v(5, 4)': 0, 'v(5, 5)': 1, 'v(6, 1)': 0, 'v(6, 2)': 1, 'v(6, 3)': 0, 'v(6, 4)': 1, 'v(6, 5)': 0, 'v(7, 1)': 0, 'v(7, 2)': 0, 'v(7, 3)': 1, 'v(7, 4)': 1, 'v(7, 5)': 0, 'v(8, 1)': 0, 'v(8, 2)': 0, 'v(8, 3)': 0, 'v(8, 4)': 0, 'v(8, 5)': 0, 'v(9, 1)': 0, 'v(9, 2)': 0, 'v(9, 3)': 0, 'v(9, 4)': 0, 'v(9, 5)': 0, 'x1': 0, 'x10': 0, 'x11': 1, 'x12': 1, 'x13': 1, 'x14': 1, 'x15': 1, 'x16': 1, 'x17': 1, 'x18': 1, 'x19': 1, 'x2': 1, 'x20': 1, 'x21': 0, 'x22': 1, 'x23': 1, 'x24': 0, 'x25': 0, 'x26': 1, 'x27': 0, 'x28': 1, 'x29': 1, 'x3': 0, 'x30': 1, 'x31': 1, 'x32': 1, 'x33': 1, 'x34': 0, 'x35': 1, 'x36': 0, 'x37': 0, 'x38': 0, 'x39': 1, 'x4': 1, 'x5

<ipython-input-27-bc713edfb560>:1: DeprecationWarning: SampleSet.samples() will return an iterable not an iterator in the future
  first_sample = next(result.samples())


Luego para la lista hemos obtenido como sol optima:

$'x1'= 0, 'x10'= 0, 'x11'= 1, 'x12'= 1, 'x13'= 1, 'x14'= 1, 'x15'= 1, 'x16'= 1, 'x17'= 1, 'x18'= 1, 'x19'= 1, 'x2'= 1, 'x20'= 1, 'x21'= 0, 'x22'= 1, 'x23'= 1, 'x24'= 0, 'x25'= 0, 'x26'= 1, 'x27'= 0, 'x28'= 1, 'x29'= 1, 'x3'= 0, 'x30'= 1, 'x31'= 1, 'x32'= 1, 'x33'= 1, 'x34'= 0, 'x35'= 1, 'x36'= 0, 'x37'= 0, 'x38'= 0, 'x39'= 1, 'x4'= 1, 'x5'= 0, 'x6'= 0, 'x7'= 0, 'x8'= 1, 'x9'= 0$

$v_j \neq 0 ∀ j=1..m$

Con el valor de la funcion objetivo -(-9776.0) = 9776.0

Entonces la diferencia con la solucion optima es de:

In [ ]:
solopti - 9776.0

842.0

## Resultados septimo problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  #return EmbeddingComposite(DWaveSampler()) #dwave composer que te adapta el grafo a la topologia de los procesadores de dwave
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=1000)
print(result)

    v(1, 1) v(1, 2) v(1, 3) v(1, 4) v(1, 5) v(10, 1) ... x9     energy num_oc.
468       1       0       1       0       1        0 ...  0   -15012.0       1
145       0       0       1       1       0        0 ...  1   -14954.0       1
463       1       0       0       1       0        0 ...  1   -14752.0       1
502       1       0       1       1       1        0 ...  0   -14643.0       1
386       1       0       0       0       0        0 ...  0   -14595.0       1
928       1       1       0       0       1        0 ...  1   -14540.0       1
572       0       0       1       1       1        0 ...  1   -14438.0       1
994       1       0       1       1       0        0 ...  0   -14264.0       1
43        0       1       0       1       0        0 ...  1   -14126.0       1
854       0       1       0       0       0        0 ...  1   -13927.0       1
12        1       0       0       1       0        0 ...  1   -13565.0       1
547       1       1       0       1       1        0

In [ ]:
first_sample = next(result.samples())
print(first_sample)

{'v(1, 1)': 1, 'v(1, 2)': 0, 'v(1, 3)': 1, 'v(1, 4)': 0, 'v(1, 5)': 1, 'v(10, 1)': 0, 'v(10, 2)': 0, 'v(10, 3)': 0, 'v(10, 4)': 0, 'v(10, 5)': 0, 'v(2, 1)': 0, 'v(2, 2)': 0, 'v(2, 3)': 0, 'v(2, 4)': 1, 'v(2, 5)': 0, 'v(3, 1)': 0, 'v(3, 2)': 0, 'v(3, 3)': 0, 'v(3, 4)': 0, 'v(3, 5)': 0, 'v(4, 1)': 1, 'v(4, 2)': 0, 'v(4, 3)': 0, 'v(4, 4)': 1, 'v(4, 5)': 0, 'v(5, 1)': 0, 'v(5, 2)': 1, 'v(5, 3)': 0, 'v(5, 4)': 1, 'v(5, 5)': 1, 'v(6, 1)': 0, 'v(6, 2)': 0, 'v(6, 3)': 1, 'v(6, 4)': 1, 'v(6, 5)': 1, 'v(7, 1)': 0, 'v(7, 2)': 0, 'v(7, 3)': 1, 'v(7, 4)': 1, 'v(7, 5)': 0, 'v(8, 1)': 0, 'v(8, 2)': 0, 'v(8, 3)': 0, 'v(8, 4)': 0, 'v(8, 5)': 0, 'v(9, 1)': 0, 'v(9, 2)': 0, 'v(9, 3)': 0, 'v(9, 4)': 0, 'v(9, 5)': 0, 'x1': 0, 'x10': 1, 'x11': 1, 'x12': 0, 'x13': 1, 'x14': 0, 'x15': 0, 'x16': 1, 'x17': 1, 'x18': 0, 'x19': 0, 'x2': 1, 'x20': 0, 'x21': 0, 'x22': 0, 'x23': 1, 'x24': 0, 'x25': 0, 'x26': 0, 'x27': 0, 'x28': 0, 'x29': 0, 'x3': 0, 'x30': 1, 'x31': 1, 'x32': 1, 'x33': 0, 'x34': 1, 'x35': 1, 'x36': 

<ipython-input-35-bc713edfb560>:1: DeprecationWarning: SampleSet.samples() will return an iterable not an iterator in the future
  first_sample = next(result.samples())


Luego para la lista hemos obtenido como sol optima:

$'x1'= 0, 'x10'= 1, 'x11'= 1, 'x12'= 0, 'x13'= 1, 'x14'= 0, 'x15'= 0, 'x16'= 1, 'x17'= 1, 'x18'= 0, 'x19'= 0, 'x2'= 1, 'x20'= 0, 'x21'= 0, 'x22'= 0, 'x23'= 1, 'x24'= 0, 'x25'= 0, 'x26'= 0, 'x27'= 0, 'x28'= 0, 'x29'= 0, 'x3'= 0, 'x30'= 1, 'x31'= 1, 'x32'= 1, 'x33'= 0, 'x34'= 1, 'x35'= 1, 'x36'= 0, 'x37'= 0, 'x38'= 0, 'x39'= 0, 'x4'= 1, 'x40'= 1, 'x41'= 1, 'x42'= 0, 'x43'= 1, 'x44'= 1, 'x45'= 1, 'x46'= 0, 'x47'= 1, 'x48'= 0, 'x49'= 1, 'x5'= 0, 'x50'= 0, 'x6'= 1, 'x7'= 1, 'x8'= 0, 'x9'= 0$

$v_j \neq 0 ∀ j=1..m$

Con el valor de la funcion objetivo -(-15012.0) = 15012.0

Entonces la diferencia con la solucion optima es de:

In [ ]:
solopti - 15012.0

1525.0

## Prueba con un procesador cuántico, para el primer problema.

In [ ]:
def get_sampler():
  #return ExactSolver()
  #return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  ## return EmbeddingComposite(DWaveSampler()) #LO HE COMENTADO PARA NO MANDARLO DE NUEVO

In [ ]:
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) ... x6     energy num_oc. ...
80       0        0       1       1       1 ...  1  5356588.0       1 ...
16       0        1       1       0       1 ...  0  5773549.0       1 ...
17       0        0       1       0       0 ...  1  5785051.0       1 ...
18       1        1       1       0       0 ...  1  5961777.0       1 ...
3        0        0       1       1       0 ...  0  6052290.0       1 ...
21       1        1       1       0       0 ...  1  6730990.0       1 ...
23       1        1       1       0       0 ...  0  7370484.0       1 ...
75       0        1       1       1       0 ...  0  7474299.0       1 ...
25       1        0       0       1       1 ...  0  7813249.0       1 ...
7        0        0       0       1       1 ...  0  7867557.0       1 ...
36       0        1       1       1       0 ...  0  7900962.0       1 ...
89       0        0       1       1       0 ...  1  8015179.0       1 ...
24       0        1       0       1   

*Observamos que el ordenador de D-Wave no ha conseguido obtener la solucion optima del problema QUBO a diferencia de la resolucion con el clásico.

## Prueba con un procesador cuántico, para el segundo problema.

In [ ]:
def get_sampler():
  #return ExactSolver()
  #return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  ## return EmbeddingComposite(DWaveSampler()) #LO HE COMENTADO PARA NO MANDARLO DE NUEVO
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 10) v(1, 2) v(1, 3) ... x9            energy num_oc. ...
68       1        1       0       1 ...  0       747631970.0       1 ...
31       0        1       1       0 ...  1       819525248.4       1 ...
94       0        1       1       0 ...  1      1017887625.5       1 ...
61       0        0       1       0 ...  0 1052032971.400002       1 ...
96       0        0       0       0 ...  1 1054202519.299999       1 ...
83       0        1       1       1 ...  0      1058474543.0       1 ...
51       1        0       0       0 ...  0      1130731280.0       1 ...
79       1        0       0       0 ...  0 1263456841.400002       1 ...
66       0        0       1       0 ...  0      1571131280.0       1 ...
88       0        1       0       0 ...  0      1571508980.0       1 ...
40       1        1       0       0 ...  0      1579138520.0       1 ...
91       0        1       1       0 ...  0      1605262770.0       1 ...
84       0        0       0       0 ...  0 17049335

## Prueba con un procesador cuántico, para el tercer problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  #return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  ##return EmbeddingComposite(DWaveSampler()) #LO HE COMENTADO PARA NO MANDARLO DE NUEVO
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) ... x9       energy num_oc. ...
49       0        1       0       0       0 ...  0  111778933.0       1 ...
13       1        0       1       0       1 ...  1  120064847.0       1 ...
44       0        1       0       0       1 ...  1  164932277.0       1 ...
2        1        0       0       1       1 ...  1  204250553.0       1 ...
48       0        1       0       0       1 ...  1  205174407.0       1 ...
93       1        0       0       1       1 ...  1  213193109.0       1 ...
96       1        0       0       0       0 ...  1  218285865.0       1 ...
31       0        0       0       1       1 ...  1  234760949.0       1 ...
70       1        1       1       1       1 ...  1  247242765.0       1 ...
1        0        0       1       0       0 ...  1  250542839.0       1 ...
50       1        0       0       0       0 ...  1  264879114.0       1 ...
3        0        1       1       0       1 ...  1  271190836.0       1 ...
62       1  

## Prueba con un procesador cuántico, para el cuarto problema.

In [ ]:
def get_sampler():
  #return ExactSolver()
  #return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  ##return EmbeddingComposite(DWaveSampler()) #LO HE COMENTADO PARA NO MANDARLO DE NUEVO
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) ... x9       energy num_oc. ...
54       1        1       0       0       0 ...  1  265571136.0       1 ...
96       1        0       1       0       1 ...  0  309621335.0       1 ...
4        0        1       0       1       0 ...  1  382781407.0       1 ...
85       0        0       1       1       1 ...  1  390563016.0       1 ...
35       0        1       0       1       0 ...  0  411817431.0       1 ...
31       0        1       0       1       0 ...  1  412228944.0       1 ...
89       0        0       1       0       1 ...  0  421205791.0       1 ...
72       0        0       1       1       0 ...  1  432962204.0       1 ...
68       1        0       1       0       0 ...  0  434674422.0       1 ...
24       1        0       1       0       0 ...  1  473515344.0       1 ...
47       0        1       0       0       0 ...  1  543523597.0       1 ...
65       1        0       0       0       0 ...  1  567042384.0       1 ...
25       1  

## Prueba con un procesador cuántico, para el quinto problema.

In [ ]:
def get_sampler():
  #return ExactSolver()
  #return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  ##return EmbeddingComposite(DWaveSampler()) #LO HE COMENTADO PARA NO MANDARLO DE NUEVO
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 10) v(1, 2) v(1, 3) v(1, 4) ... x9        energy num_oc. ...
11       0        0       0       0       0 ...  0  1152134316.0       1 ...
66       0        0       0       0       0 ...  1  1526050616.0       1 ...
78       1        0       0       1       1 ...  0  1597657532.0       1 ...
37       0        0       0       1       0 ...  1  1639854905.0       1 ...
74       0        0       0       0       1 ...  0  1896592118.0       1 ...
43       0        0       1       1       0 ...  0  1966377476.0       1 ...
12       1        1       0       0       1 ...  0  1966896730.0       1 ...
1        1        0       1       1       0 ...  0  1970280918.0       1 ...
16       0        0       1       1       1 ...  1  1996402104.0       1 ...
70       1        1       1       1       1 ...  0  2012006156.0       1 ...
34       0        1       0       1       1 ...  0  2140889773.0       1 ...
81       0        1       1       1       0 ...  0  2154012750.0       1 ...

## Prueba con un procesador cuántico, para el sexto problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  #return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  ##return EmbeddingComposite(DWaveSampler()) #LO HE COMENTADO PARA NO MANDARLO DE NUEVO
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 2) v(1, 3) v(1, 4) v(1, 5) ... x9       energy num_oc. ...
63       1       1       0       0       0 ...  1  283795425.0       1 ...
3        1       0       0       0       0 ...  1  372080778.0       1 ...
56       1       1       0       1       0 ...  0  376321487.0       1 ...
83       1       1       1       0       1 ...  0  582946297.0       1 ...
41       1       1       1       1       0 ...  0  638837318.0       1 ...
79       1       1       0       0       1 ...  0  666400148.0       1 ...
58       1       1       1       1       0 ...  0  696997862.0       1 ...
99       1       1       0       0       1 ...  0  747104196.0       1 ...
19       0       0       1       1       1 ...  0  846061874.0       1 ...
27       1       1       1       0       0 ...  0  920685509.0       1 ...
72       1       1       0       0       0 ...  1  972249410.0       1 ...
57       1       0       1       0       0 ...  1  976842158.0       1 ...
14       0       0       

## Prueba con un procesador cuántico, para el septimo problema

In [ ]:
def get_sampler():
  #return ExactSolver()
  #return SimulatedAnnealingSampler()
  #return SteepestDescentSampler()
  ##return EmbeddingComposite(DWaveSampler()) #LO HE COMENTADO PARA NO MANDARLO DE NUEVO
sampler = get_sampler()
result = sampler.sample(qubo, num_reads=100)
print(result)

   v(1, 1) v(1, 2) v(1, 3) v(1, 4) v(1, 5) ... x9       energy num_oc. ...
60       0       1       0       0       1 ...  1   76885348.0       1 ...
83       1       1       0       0       1 ...  1  153626247.0       1 ...
89       0       0       0       1       0 ...  0  346636824.0       1 ...
49       0       0       0       0       1 ...  0  368818275.0       1 ...
62       1       1       1       0       0 ...  1  600300720.0       1 ...
69       0       0       1       0       0 ...  1  646918442.0       1 ...
8        0       0       0       0       0 ...  0  769330121.0       1 ...
98       0       0       0       1       1 ...  1  780486790.0       1 ...
55       0       0       0       0       0 ...  1  957211635.0       1 ...
72       1       0       0       1       1 ...  1 1008325437.0       1 ...
50       0       0       0       1       1 ...  0 1189794578.0       1 ...
77       0       1       0       0       1 ...  1 1287277593.0       1 ...
14       0       0       



---



Trabajo hecho por Manuel Enciso Martinez



---

